In [1]:
from __future__ import annotations
import requests
from dotenv import load_dotenv
import os, re, json, requests, time, argparse
from dataclasses import dataclass
from typing import Dict, Any, Callable, Tuple
from langchain import hub

load_dotenv()

True

In [2]:
prompt = hub.pull("hwchase17/structured-chat-agent")

print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] optional_variables=['chat_history'] input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag

In [ ]:
# ------------------------------
# Tools
# ------------------------------
def tool_calc(query: str) -> str:
    """Safe arithmetic (+,-,*,/,**,(), integer/float)."""
    allowed = re.compile(r"^[\d\.\+\-\*\/\(\)\s\^eE]+$")
    q = query.replace("^", "**")
    if not allowed.match(q):
        return "Error: only arithmetic is allowed."
    try:
        # Very small sandbox: eval with empty globals/locals
        result = eval(q, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {e}"

def tool_rag(query: str, api_url: str) -> str:
    """Call your existing RAG API: POST /ask -> {answer, sources[]}"""
    try:
        r = requests.post(api_url, json={"question": query}, timeout=60)
        r.raise_for_status()
        data = r.json()
        ans = data.get("answer", "")
        srcs = data.get("sources", [])
        if srcs:
            src_str = "; ".join([f"{s.get('doc','?')}#{s.get('section_id','?')}" for s in srcs])
            return f"{ans}\n[근거] {src_str}"
        return ans or "(no answer)"
    except Exception as e:
        return f"RAG error: {e}"

@dataclass
class ToolSpec:
    name: str
    description: str
    func: Callable[[str], str]

def build_tool_registry(rag_api: str | None) -> Dict[str, ToolSpec]:
    reg = {
        "calc": ToolSpec(
            name="calc",
            description="산술 계산기 (+,-,*,/,^, 괄호 지원). 입력에는 계산식만 넣어.",
            func=tool_calc,
        )
    }
    if rag_api:
        reg["rag"] = ToolSpec(
            name="rag",
            description="사내 문서 기반 질문에 답하기. 입력: 자연어 질문.",
            func=lambda q: tool_rag(q, rag_api),
        )
    return reg

In [ ]:
SYS_PROMPT = """\
너는 실용적인 도우미 Agent야. 네가 직접 모든 걸 알 필요는 없어.
다음 '도구' 목록 중에서 필요한 도구를 골라 단계적으로 문제를 해결해.
반드시 다음 형식을 따를 것:

Thought: (현재 문제를 어떻게 풀지 생각)
Action: <도구 이름 중 하나>
Action Input: <도구에 전달할 입력 텍스트>
Observation: <도구 실행 결과를 여기에 적음>

... (필요하면 여러 번 반복) ...

마지막에는 다음 형식으로 답변을 종료해:
Final Answer: <사용자에게 줄 최종 답변>

주의:
- 모르면 도구(rag)를 사용하여 확인하거나, 모른다고 말해.
- 환각(근거 없는 내용) 금지. 근거가 없으면 '모르겠다'고 해.
- 답변은 한국어로.
"""

TOOL_FMT_HEADER = "사용 가능한 도구 목록:\n"
TOOL_FMT_ITEM = "- {name}: {desc}\n"

USER_WRAP = "사용자 질문: {q}"

In [ ]:
ACTION_RE = re.compile(r"Action:\s*(\w+)\s*", re.IGNORECASE)
ACTION_INPUT_RE = re.compile(r"Action Input:\s*(.*)", re.IGNORECASE)

def run_agent(tools: Dict[str, ToolSpec], question: str, max_steps=4) -> str:
    tool_desc = TOOL_FMT_HEADER + "".join(TOOL_FMT_ITEM.format(name=k, desc=v.description) for k, v in tools.items())
    scratch = f"{SYS_PROMPT}\n{tool_desc}\n{USER_WRAP.format(q=question)}\n"
    print(scratch)

In [ ]:
ap = argparse.ArgumentParser()

In [ ]:
ap.add_argument("--rag_api", type=str, default=None, help="e.g., http://localhost:8000/ask")

In [ ]:
ap.add_argument("--max_steps", type=int, default=4)
args = ap.parse_args([])  # set to [] for notebook; remove for real CLI

In [ ]:
args.rag_api

In [ ]:
tools = build_tool_registry("yes")

In [ ]:
tools

In [ ]:
run_agent(tools, "hi")

In [4]:
from mcp_use import MCPAgent, MCPClient
from openai import OpenAI

In [ ]:
config = {
    "mcpServers": {
        "name": "demo",
        "transport": "stdio",
        "command": "uv",
        "args": ["run", "python", "src/mcp_test_server.py", "stdio"]
    }
}

In [ ]:
# Create MCPClient from config file
client = MCPClient.from_dict(config)

In [8]:
llm_client = OpenAI(
    base_url = os.getenv("ollama_url") + "v1",
    api_key='ollama', # required, but unused
)

In [9]:
response = llm_client.chat.completions.create(
  model=os.getenv("ollama_model"),
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

The 2020 World Series was played in **Boston**! Specifically, it was held at Fenway Park, though it was mostly played in empty stadiums due to the pandemic.


In [10]:
agent = MCPAgent(
    llm={"type": "openai", "client": llm_client, "model": os.getenv("ollama_model")},
    client=client
)

2025-09-12 11:26:39,339 - mcp_use.telemetry.telemetry - INFO - Anonymized telemetry enabled. Set MCP_USE_ANONYMIZED_TELEMETRY=false to disable.


In [11]:
response = MCPAgent.run(self=agent, query="2랑 3 더해봐.", max_steps=5)
response

<coroutine object MCPAgent.run at 0x0000023AA5B71240>

In [12]:
response.send(None)

2025-09-12 11:26:52,846 - mcp_use - INFO - 🚀 Initializing MCP agent and connecting to services...
2025-09-12 11:26:52,847 - mcp_use - ERROR - ❌ Error running query: 'OpenAI' object has no attribute 'get_all_active_sessions'
2025-09-12 11:26:52,847 - mcp_use - ERROR - ❌ Error during agent execution: 'OpenAI' object has no attribute 'get_all_active_sessions'


AttributeError: 'OpenAI' object has no attribute 'get_all_active_sessions'